In [1]:
import torch
import torch.nn as nn

C:\Users\radcl\AppData\Roaming\Python\Python313\site-packages\torch\_subclasses\functional_tensor.py:279: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_numpy.cpp:81.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


In [2]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,    # Vocabulary size
    "context_length": 1024, # Context length
    "emb_dim": 768,         # Embedding dimension
    "n_heads": 12,          # Number of attention heads
    "n_layers": 12,         # Number of layers
    "drop_rate": 0.1,       # Dropout rate
    "qkv_bias": False       # Query-Key-Value bias
}

In [3]:
class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift

In [4]:
class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) * 
            (x + 0.044715 * torch.pow(x, 3))
        ))

class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
            GELU(),
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),
        )

    def forward(self, x):
        return self.layers(x)

In [5]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        # As in `CausalAttention`, for inputs where `num_tokens` exceeds `context_length`, 
        # this will result in errors in the mask creation further below. 
        # In practice, this is not a problem since the LLM (chapters 4-7) ensures that inputs  
        # do not exceed `context_length` before reaching this forward method.

        keys = self.W_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) 
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)
        
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2) 
        
        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec

In [6]:
class TransformerBlock( nn.Module ):
    def __init__( self, cfg ):
        super().__init__()
        self.norm1 = LayerNorm( cfg["emb_dim"] )
        self.att = MultiHeadAttention( 
            d_in = cfg["emb_dim"], 
            d_out = cfg["emb_dim"], 
            context_length = cfg["context_length"], 
            dropout = cfg["drop_rate"], 
            num_heads = cfg["n_heads"] 
        )
        self.dropout = nn.Dropout( cfg["drop_rate"] )
        self.norm2 = LayerNorm( cfg["emb_dim"] )
        self.feedforward = FeedForward( cfg )
        
    def forward( self, x ):
        
        # the attention chunk in this transformer:
        shortcut = x 
        x = self.norm1( x )
        x = self.att( x )
        x = self.dropout( x )
        x = x + shortcut 
        
        # the feed forward chunk:
        shortcut = x
        x = self.norm2( x ) 
        x = self.feedforward( x )
        x = self.dropout( x )
        x = x + shortcut 
        
        return x 

In [7]:
# instantiate TransformerBlock
block = TransformerBlock( GPT_CONFIG_124M )

In [8]:
input = torch.randn( 3, 4, 768 )

In [9]:
input

tensor([[[ 0.8412,  0.5740,  0.9288,  ...,  0.0397, -0.1103,  0.1566],
         [ 0.5393,  0.1277, -0.4722,  ...,  1.5074, -0.2563, -0.7534],
         [-0.0946, -1.1412,  0.3639,  ...,  0.3860, -0.5217,  1.6289],
         [-0.2142,  0.6457, -0.0034,  ...,  0.2486,  0.1273,  0.2603]],

        [[-0.5208, -0.7901,  0.6932,  ...,  1.6494, -0.3308, -0.3748],
         [-1.0183, -0.8135,  0.0027,  ...,  0.5673, -0.6288,  0.5569],
         [ 1.0712, -0.1431, -0.7707,  ...,  0.6954,  0.6179,  1.0784],
         [ 0.3172,  0.9707, -0.2583,  ...,  0.4894, -0.0917, -1.4419]],

        [[ 0.9309,  0.4920,  0.3297,  ...,  0.8089,  0.6557, -1.6920],
         [ 1.1947,  2.2224, -0.3645,  ...,  0.8795,  0.1671,  0.7725],
         [-1.6223, -1.1554,  0.6455,  ...,  0.6789, -0.2742, -0.8268],
         [-1.4056, -1.0217,  0.4838,  ...,  0.3119, -0.6941, -1.1740]]])

In [10]:
output = block( input )

In [11]:
output.shape

torch.Size([3, 4, 768])

In [12]:
output

tensor([[[ 0.2925,  0.5407,  0.7754,  ...,  0.2318, -0.3114, -0.7634],
         [-0.1713,  0.6341, -0.5496,  ...,  1.1718, -0.5684, -1.2127],
         [-0.4100, -0.3861,  0.0977,  ...,  0.1854, -0.6987,  1.5552],
         [-0.3992,  1.0261,  0.1100,  ...,  0.1047,  0.2743,  0.1669]],

        [[-1.3870, -0.4091,  0.5648,  ...,  1.0654, -0.3961, -0.5752],
         [-1.0976, -1.0997,  0.1613,  ...,  0.1107, -0.1105,  0.0313],
         [ 0.9448, -0.0322, -0.8919,  ...,  0.9133,  0.4822,  0.7794],
         [ 0.2869,  1.7208, -0.3127,  ...,  0.2199, -0.0980, -1.9762]],

        [[ 0.5018,  0.7173,  0.4991,  ...,  0.5382,  0.1210, -1.5143],
         [ 0.1521,  2.0974, -0.8303,  ...,  0.7778,  0.7744,  0.8849],
         [-1.9898, -1.4537,  1.0638,  ...,  0.7772, -0.8437, -0.5069],
         [-1.4015, -1.2034,  0.3974,  ...,  0.5778, -0.6309, -1.2344]]],
       grad_fn=<AddBackward0>)